# Comic Book Scraper For Redo (Final)

   At work recently, I had the task of pricing 14 long boxes of comics in two days. Each long box contains approximately 275 comics that my boss wanted cataloged and priced. Since I was the only working on this assignment, I needed a way to quickly and effectively price this collection before the sale. I then realized I could use my skills in Python to make this assignment a lot easier. I found a site, mycomicbookshop.com, that conveniently provides a price and an "Add to Cart Option" for most of the comics in the collection. The only way I was able to complete this assignment is to add each comic to the cart and then scrape all the information I would need later, as opposed to manually entering it into Excel. This code is a sample of about 600 of those comics, and I thought it would be interesting to perform an analysis with this information.

Some things to note before continuing: 

1. The owner of this collection of comics is not comfortable with the contents of his entire collection being posted online at this time, so this project will only be on the contents of approximately three long boxes selected at random.
2. This version solves the error when the program encounters a 503-gateway error on select pages.
3. Simple visualizations will be performed, but the bulk of the EDA will be done in Tableau.
4. An explanation for the comic book grading system can be found here, https://www.mycomicshop.com/help/grading, for those unfamiliar with how the grading system works.
5. After conversing with some people in the comic collecting community about this project, I have been informed that the prices on mycomicbookshop.com are slightly overpriced. To compensate this we took 25% off the calculated price in addition to the discount we gave on the entire lot.
6. I had some help writing this code, and I would like to thank user doctorevil92 for helping me get past the 503 gateway error and helping me out when I struggled with overcoming t. I could not have done this without him and I am extremely grateful for his help.

## Data Extraction and Cleaning from mycomicbookshop.com

In [ ]:
USERNAME = "myusername"
PASSWORD = "mypassword"

In [ ]:
from selenium import webdriver  #install selenium
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import pprint
import os
import sys
from datetime import datetime
from lxml import html # pip install lxml
import csv
import sqlite3

In [ ]:
TIME_PAUSE = 1.0 # pause

def wait_by_xpath(xp, how_long_to_wait): # xp is string, how_long_to_wait float - the number of seconds to wait
    try:
        WebDriverWait(driver, how_long_to_wait).until(EC.presence_of_element_located((By.XPATH, xp)) )
        time.sleep(TIME_PAUSE)
        return 1 # success
    except TimeoutException:
        print ("Too much time has passed while waiting for", xp)
        return 0 # fail

In [ ]:
def fix_string(entry_string): # remove "\n", "\t" and double spaces
    exit_string = entry_string.replace("\n", "")
    exit_string = exit_string.replace("\t", "")
    exit_string = exit_string.replace("\r", "")
    while "  " in exit_string:
        exit_string = exit_string.replace("  ", " ")
    if len(exit_string) > 0: # remove first space
        if exit_string[0] == ' ':
            exit_string = exit_string[1:len(exit_string)]
    if len(exit_string) > 0: # remove last space
        if exit_string[len(exit_string)-1] == ' ':
            exit_string = exit_string[0:len(exit_string)-1]

    return exit_string

In [ ]:
def PARSE(input_url, input_name, input_price, input_html):
    data_to_return = {}
    data_to_return["URL"] = input_url
    data_to_return["Full Price"] = input_price

    tree = html.document_fromstring(input_html)

    data_to_return["Title"] = ''
    data_to_return["Series"] = ''
    data_to_return["Condition"] = ''
    #title_el = tree.xpath("//div[@class='primary']/h1[@class='top']")
    #if len(title_el) != 0:
    full_title_text = input_name
    data_to_return["Title"] = full_title_text

    if "(" in full_title_text and ")" in full_title_text:
        if full_title_text.find("(") < full_title_text.find(")"):
            data_to_return["Series"] = full_title_text[full_title_text.find("(")+1 : full_title_text.find(")")]

            data_to_return["Condition"] = fix_string("".join(char for char in full_title_text[full_title_text.find(")")+1 : len(full_title_text)] if not char.isdigit()))

    
    data_to_return["Month Published"] = ''
    data_to_return["Year Published"] = ''
    published_el = tree.xpath("//div[@class='primary']//div/text()[contains(., 'Published')]/following-sibling::a[@href][1]")
    if len(published_el) != 0:
        published_time_fullstring = fix_string(published_el[0].text_content())
        published_time_array = [pub_item for pub_item in published_time_fullstring.split(" ") if pub_item!='']
        if len(published_time_array) == 1:
            # probably year only
            try:
                probe_year = int(published_time_array[0])
                data_to_return["Year Published"] = probe_year
            except ValueError:
                print ("Unable to parse year of publishing with 1 element on URL", input_url)
                pass

        elif len(published_time_array) == 2:
            # should be month and year
            try:
                probe_year = int(published_time_array[1])
                data_to_return["Year Published"] = probe_year
                data_to_return["Month Published"] = published_time_array[0]
            except ValueError:
                print ("Unable to parse year of publishing with 2 elements on URL", input_url)
                pass
        else:
            print ("Unknown form of date for", published_time_array)


    data_to_return["Publisher"] = ''
    publisher_el = tree.xpath("//div[@class='primary']//div/text()[contains(., 'Published')]/following-sibling::text()[contains(., 'by')]/following-sibling::a[@href][1]")
    if len(publisher_el) != 0:
        data_to_return["Publisher"] = publisher_el[0].text_content()

    data_to_return["Tags"] = ''
    tags_els = tree.xpath("//div[@class='primary']//div[@class='title']/following-sibling::div/text()[contains(., 'Tags')]/following-sibling::a[@href]")
    for tag_el in tags_els:
        data_to_return["Tags"] = data_to_return["Tags"] + tag_el.text_content() + ", "
    data_to_return["Tags"] = data_to_return["Tags"][0:data_to_return["Tags"].rfind(", ")]

    data_to_return["Description"] = ''
    descr_el = tree.xpath("//div[@class='primary']//div[@class='tabcontents']/div[@class='issuegrades']/following-sibling::p[1]/text()")
    if len(descr_el) != 0:
        data_to_return["Description"] = descr_el[0]

    data_to_return["Cover Price"] = '' # get from description text
    if len(descr_el) != 0:
        descr_to_parse = descr_el[0].lower()
        cover_price_text = fix_string(descr_to_parse[descr_to_parse.rfind("cover price")+len("cover price") : descr_to_parse.rfind(".")])
        try:
            probe_price = float(cover_price_text.replace("$", ""))
            data_to_return["Cover Price"] = cover_price_text
        except ValueError:
            pass
    
    return data_to_return

In [ ]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
driver.maximize_window()

In [ ]:
### first, login
try:
    driver.get("https://www.mycomicshop.com/login")
except:
    print ("Some kind of exception, quit!")
    driver.quit()
    sys.exit(0)

    
try:
    username_el = driver.find_element_by_xpath("//input[@name='CustomerEmail']")
    username_el.send_keys(USERNAME)
    password_el = driver.find_element_by_xpath("//input[@name='CustomerPassword']")
    password_el.send_keys(PASSWORD)
    login_el = driver.find_element_by_xpath("//input[@type='submit' and @value='Log In']")
    driver.execute_script("arguments[0].click();", login_el)
    time.sleep(5.0)
except:
    print ("Some kind of exception while logging in, quit!")
    driver.quit()
    sys.exit(0)


try:
    driver.get("https://www.mycomicshop.com/cart")
    wait_for_cart = wait_by_xpath("//h2[text()='Saved For Later']/following-sibling::table[1]/tbody/tr/td[1]/a[contains(@href, 'search?')]/../..", 30)
    if wait_for_cart == 0:
        print ("Failed wait for cart elements, quit!")
        driver.quit()
        sys.exit(0)
    innerHTML_cart = driver.execute_script("return document.body.innerHTML")
    htmlElem_cart = html.document_fromstring(innerHTML_cart)
except:
    print ("Some kind of exception while opening cart link, quit!")
    driver.quit()
    sys.exit(0)
### login done

In [ ]:
### get a list of items from cart html, visit all urls that need to be visited
items_to_visit = [] # keys: name, url, price
comic_els = htmlElem_cart.xpath("//h2[text()='Saved For Later']/following-sibling::table[1]/tbody/tr/td[1]/a[contains(@href, 'search?')]/../..")
for comic_el in comic_els:
    comic_link_el = comic_el.xpath("./td[1]/a[@href]")
    comic_price_el = comic_el.xpath("./td[2]")

    if len(comic_link_el) != 0 and len(comic_price_el) != 0:
        items_to_visit.append( {"name":comic_link_el[0].text_content(), "price":fix_string(comic_price_el[0].text_content()), "url": "https://www.mycomicshop.com" + comic_link_el[0].attrib["href"]} )

# create a database, get already scraped links
db_conn = sqlite3.connect("COMICS_DB.db")
db_cursor = db_conn.cursor()
db_cursor.execute("CREATE TABLE IF NOT EXISTS ComicsData (url TEXT, name TEXT, price TEXT, html TEXT)")

already_scraped_links = {}
for already_scraped_item in db_cursor.execute("SELECT url FROM ComicsData"):
    already_scraped_links[already_scraped_item[0]] = ''

# now visit every comic url and save html if some kind of verificator is found.
for item_to_visit in items_to_visit:
    if item_to_visit["url"] in already_scraped_links:
        continue # this was already scraped

    # if here, try to open link
    try:
        driver.get(item_to_visit["url"])
        wait_for_verificator = wait_by_xpath("//div[@class='primary']/h1[@class='top']", 20)
        if wait_for_verificator == 0:
            continue
        innerHTML_cart = driver.execute_script("return document.body.innerHTML")
        htmlElem_cart = html.document_fromstring(innerHTML_cart)

    except KeyboardInterrupt:
        print ("Manual interrupt, quit everything!")
        driver.quit()
        db_cursor.close()
        db_conn.close()
        sys.exit(0)

    except:
        time.sleep(2.0)
        continue

    #if still here, save html, update scraped links and continue to the next one.
    db_cursor.execute("INSERT INTO ComicsData (url, name, price, html) VALUES (?,?,?,?)",
                  (item_to_visit["url"], item_to_visit["name"], item_to_visit["price"], innerHTML_cart) ) # not necessary to insert all columns!
    db_conn.commit()
    already_scraped_links[item_to_visit["url"]] = ''
    print ("Scraped link number", items_to_visit.index(item_to_visit)+1, "/", len(items_to_visit))

driver.quit()

In [ ]:
### parse for all items in items_to_visit!
print ("Parsing listed comics...")
headers_in_order = ['Title', 'Series', 'Condition', 'Publisher', 'Month Published', 'Year Published', 'Tags', 'Description', 'Cover Price', 'Full Price', 'URL']
output_filename = datetime.now().strftime("%m-%d-%Y %H_%M_%S") + ".csv"
with open(output_filename, "w", newline="", encoding="utf-8") as csvfil:
    writer = csv.writer(csvfil, delimiter=";", quoting=csv.QUOTE_MINIMAL)
    writer.writerow(headers_in_order) # write headers
    
    for item_to_fetch in items_to_visit:
        # for each link, get data from database
        for fetched_data in db_cursor.execute("SELECT * FROM ComicsData WHERE url=?", (item_to_fetch["url"],)):
            parsed_data = PARSE(fetched_data[0], fetched_data[1], fetched_data[2], fetched_data[3])
            # write from parsed_data to csv
            row_to_write = [parsed_data[key_to_write] for key_to_write in headers_in_order]
            writer.writerow(row_to_write)
            break # only 1 result
        
print ("Output file:", output_filename)    
db_cursor.close()
db_conn.close()